In [147]:
import pandas as pd
import os
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, pipeline
import math
import torch
import numpy as np

input_dir = '../data/inputs'
output_dir = '../data/outputs/s0'
results_dir = '../data/results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [148]:
model_str_map = {
    'llama3-8b': 'Llama3 8B',
    'llama3-8b-instruct': 'Llama3 8B (Instruct)',
    'mistral-7b': 'Mistral 7B',
    'mistral-7b-instruct': 'Mistral 7B (Instruct)',
    'llama2-7b': 'Llama2 7B',
    'llama2-7b-instruct': 'Llama2 7B (Instruct)',
}
model_strs = ['llama3-8b', 'llama3-8b-instruct', 'mistral-7b', 'mistral-7b-instruct', 'llama2-7b', 'llama2-7b-instruct']
model_strs = sorted(model_strs, key=len, reverse=True)

In [149]:
female_ratios = pd.read_csv(os.path.join(input_dir, 'female_ratios.csv'))
# for model in  llama3-8b llama3-8b-instruct mistral-7b mistral-7b-instruct llama2-7b llama2-7b-chat


prompt_ids = ['none','low-1','low-2','medium-3','medium-4','high-5','high-6']
prompt_id_mapping = {pid: idx for idx, pid in enumerate(prompt_ids)}

df_list = []
for f in os.listdir(output_dir):
    if "conv" not in f and "gender" not in f and 'gpt2' not in f:
    # if 'conversation.csv' in f:
    
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])

        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['debiasing_prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)


        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        if 'male_met-met' in df.columns:
            col_ends = ['met-met', 'friend', 'talk-met']
            # Compute averages for male, female, and diverse columns
            male_cols = ['male_' + end for end in col_ends]
            female_cols = ['female_' + end for end in col_ends]
            diverse_cols = ['diverse_' + end for end in col_ends]

    
            grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
            grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
            grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
            grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)

            
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'implicit.csv'), index=False)
        

In [150]:
df_list = []
for f in os.listdir(output_dir):
    if 'genderquestion.csv' in f and 'gpt2' not in f:
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])


        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['debiasing_prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)

        
        # grouped_df = df.groupby('female_dominated').mean().reset_index()
        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        if 'male_met-met' in df.columns:
            col_ends = ['met-met', 'friend', 'talk-met']
            # Compute averages for male, female, and diverse columns
            male_cols = ['male_' + end for end in col_ends]
            female_cols = ['female_' + end for end in col_ends]
            diverse_cols = ['diverse_' + end for end in col_ends]

    
            grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
            grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
            grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
            grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)

            
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'explicit.csv'), index=False)

In [151]:
df_implicit = pd.read_csv(os.path.join(results_dir, 'implicit.csv'))
df_explicit = pd.read_csv(os.path.join(results_dir, 'explicit.csv'))

# if any column starts with a space, remove it
df_implicit.columns = df_implicit.columns.str.strip()
df_explicit.columns = df_explicit.columns.str.strip()

for model, group_df in df_implicit.groupby('model'):
    # Save the DataFrame to a CSV file
    if not os.path.exists(os.path.join(results_dir,model)):
        os.makedirs(os.path.join(results_dir,model))
    group_df.to_csv(os.path.join(results_dir,model, f'implicit.csv'), index=False)

for model, group_df in df_explicit.groupby('model'):
    # Save the DataFrame to a CSV file
    if not os.path.exists(os.path.join(results_dir,model)):
        os.makedirs(os.path.join(results_dir,model))
    group_df.to_csv(os.path.join(results_dir,model, f'explicit.csv'), index=False)


In [152]:
prompt_id_map = {
    0: 'None',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6'
}

abstraction_levels = {
    0: '',
    1: 'High',
    2: 'High',
    3: 'Med.',
    4: 'Med.',
    5: 'Low',
    6: 'Low'
}

abs_order = ['', 'High', 'Med.', 'Low']

In [153]:
df_implicit = pd.read_csv(os.path.join(results_dir, model, 'implicit.csv'))

In [154]:
df_implicit

,female_dominated,model,conversation,male_met-met_prob,female_met-met_prob,diverse_met-met_prob,male_friend_prob,female_friend_prob,diverse_friend_prob,male_talk-met_prob,female_talk-met_prob,diverse_talk-met_prob,female_ratio,debiasing_prompt_id,male,female,diverse
0,False,mistral-7b-instruct,False,0.410193,0.006906,0.014445,0.894686,0.008222,0.003306,0.915289,0.029218,0.053381,5.535,0.0,0.949883,0.019478,0.030638
1,True,mistral-7b-instruct,False,0.023471,0.290500,0.026629,0.076144,0.819887,0.008868,0.228155,0.645223,0.094725,89.265,0.0,0.149688,0.777655,0.072657
2,False,mistral-7b-instruct,False,0.042350,0.007928,0.024922,0.562669,0.136326,0.008648,0.731328,0.063211,0.204945,5.535,1.0,0.661385,0.129182,0.209430
3,True,mistral-7b-instruct,False,0.002202,0.065159,0.034012,0.068567,0.517397,0.011637,0.173626,0.466581,0.319889,89.265,1.0,0.124948,0.623243,0.251817
4,False,mistral-7b-instruct,False,0.545999,0.079046,0.055542,0.782168,0.056382,0.021918,0.869246,0.071427,0.058191,5.535,2.0,0.850060,0.092508,0.057438
5,True,mistral-7b-instruct,False,0.024085,0.593067,0.047012,0.111543,0.713490,0.049710,0.263152,0.601335,0.100628,89.265,2.0,0.168500,0.752747,0.078760
6,False,mistral-7b-instruct,False,0.037073,0.034011,0.523692,0.002337,0.019565,0.897264,0.309659,0.090892,0.596866,5.535,3.0,0.127317,0.058255,0.814438
7,True,mistral-7b-instruct,False,0.006987,0.186201,0.338805,0.025522,0.036379,0.815533,0.127773,0.356118,0.482107,89.265,3.0,0.064785,0.253845,0.681373
8,False,mistral-7b-instruct,False,0.000003,0.001089,0.972595,0.000003,0.000014,0.993878,0.000127,0.000016,0.999822,5.535,5.0,0.000040,0.000380,0.999572
9,True,mistral-7b-instruct,False,0.000004,0.001019,0.959729,0.000001,0.000011,0.994698,0.000311,0.000240,0.998493,89.265,5.0,0.000100,0.000440,0.999460


In [159]:
 # Update this to your results directory
models = list(model_str_map.keys())

output_tex_file = os.path.join(results_dir, 'aggregated_results.tex')

with open(output_tex_file, 'w') as f_out:
    f_out.write(r'\onecolumn')
    for model in models:
        model_name = model.replace('-', '')
        model_description = model_str_map[model]

        f_out.write(r'\subsection{' + model_description + '}\n')
        # Load the CSV file into a DataFrame
        df_implicit = pd.read_csv(os.path.join(results_dir, model, 'implicit.csv'))
        df_explicit = pd.read_csv(os.path.join(results_dir, model, 'explicit.csv'))
        
        

        # Add the abstraction level column to the DataFrame
        df_implicit['Abs.'] = df_implicit['debiasing_prompt_id'].map(abstraction_levels)
        df_explicit['Abs.'] = df_explicit['debiasing_prompt_id'].map(abstraction_levels)

        # Calculate means for each combination of prompt_id, conversation, and female_dominated
        grouped_df_implicit = df_implicit.groupby(['Abs.', 'model', 'debiasing_prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()    
        grouped_df_explicit = df_explicit.groupby(['Abs.', 'model', 'debiasing_prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()    

        # Drop the 'model' column
        model_df_implicit = grouped_df_implicit.drop(columns=['model'])
        model_df_explicit = grouped_df_explicit.drop(columns=['model'])
        
        # Sort by Abs. and prompt_id to ensure correct order
        model_df_implicit['debiasing_prompt_id'] = model_df_implicit['debiasing_prompt_id'].astype(int)
        model_df_implicit['Abs.'] = pd.Categorical(model_df_implicit['Abs.'], categories=abs_order, ordered=True)
        model_df_implicit = model_df_implicit.sort_values(by=['Abs.', 'debiasing_prompt_id'])
        
        model_df_explicit['debiasing_prompt_id'] = model_df_explicit['debiasing_prompt_id'].astype(int)
        model_df_explicit['Abs.'] = pd.Categorical(model_df_explicit['Abs.'], categories=abs_order, ordered=True)
        model_df_explicit = model_df_explicit.sort_values(by=['Abs.', 'debiasing_prompt_id'])
        
        model_df_implicit['explicit'] = False
        model_df_explicit['explicit'] = True
        
        model_df = pd.concat([model_df_implicit, model_df_explicit])

        latex_table = r'''
\begin{table*}[ht!]
\centering
\small
% Reduce text size and slightly the gap between columns
\setlength{\tabcolsep}{4.6pt} % Default: 5pt
% \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
\begin{tabular}{l c c c c c c c c c c c c c c}
\toprule
& & \multicolumn{6}{c}{Explicit} & \multicolumn{6}{c}{Implicit} \\
\cmidrule(lr){3-8} \cmidrule(lr){9-14}
& & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
\cmidrule(lr){3-5} \cmidrule(lr){6-8} \cmidrule(lr){9-11} \cmidrule(lr){12-14}
    Abs. & ID & M & F & D & M & F & D & M & F & D & M & F & D\\
    \midrule
'''


        previous_abs_level = None

        for abs_level, abs_group in model_df.groupby('Abs.'):
            if previous_abs_level is not None:
                latex_table += "        \\midrule\n"
            for prompt_id, prompt_group in abs_group.groupby('debiasing_prompt_id'):
                id_label = 'None' if prompt_id == 0 else str(prompt_id)
                row_prefix = f"        {abs_level if prompt_id in [0,2,4,6] else ''} & {id_label} & "

                explicit_fd = prompt_group[(prompt_group['explicit'] == True) & (prompt_group['female_dominated'] == True)]
                explicit_md = prompt_group[(prompt_group['explicit'] == True) & (prompt_group['female_dominated'] == False)]
                implicit_fd = prompt_group[(prompt_group['explicit'] == False) & (prompt_group['female_dominated'] == True)]
                implicit_md = prompt_group[(prompt_group['explicit'] == False) & (prompt_group['female_dominated'] == False)]

                row = row_prefix

                if not explicit_fd.empty:
                    row += f"{explicit_fd['male'].values[0]*100:.1f}\\% & {explicit_fd['female'].values[0]*100:.1f}\\% & {explicit_fd['diverse'].values[0]*100:.1f}\\% & "
                else:
                    row += " & & & "

                if not explicit_md.empty:
                    row += f"{explicit_md['male'].values[0]*100:.1f}\\% & {explicit_md['female'].values[0]*100:.1f}\\% & {explicit_md['diverse'].values[0]*100:.1f}\\% & "
                else:
                    row += " & & & "

                if not implicit_fd.empty:
                    row += f"{implicit_fd['male'].values[0]*100:.1f}\\% & {implicit_fd['female'].values[0]*100:.1f}\\% & {implicit_fd['diverse'].values[0]*100:.1f}\\% & "
                else:
                    row += " & & & "

                if not implicit_md.empty:
                    row += f"{implicit_md['male'].values[0]*100:.1f}\\% & {implicit_md['female'].values[0]*100:.1f}\\% & {implicit_md['diverse'].values[0]*100:.1f}\\% "
                else:
                    row += " & & "

                row += r"\\"
                latex_table += row + "\n"

            if abs_level != '':  # Add averages for all except "None"
                avg_row_prefix = f"         & \\textbf{{Avg}} & "

                avg_explicit_fd = abs_group[(abs_group['explicit'] == True) & (abs_group['female_dominated'] == True)][['male', 'female', 'diverse']].mean(numeric_only=True)
                avg_explicit_md = abs_group[(abs_group['explicit'] == True) & (abs_group['female_dominated'] == False)][['male', 'female', 'diverse']].mean(numeric_only=True)
                avg_implicit_fd = abs_group[(abs_group['explicit'] == False) & (abs_group['female_dominated'] == True)][['male', 'female', 'diverse']].mean(numeric_only=True)
                avg_implicit_md = abs_group[(abs_group['explicit'] == False) & (abs_group['female_dominated'] == False)][['male', 'female', 'diverse']].mean(numeric_only=True)

                avg_row = avg_row_prefix

                if not avg_explicit_fd.empty:
                    avg_row += f"\\textbf{{{avg_explicit_fd['male']*100:.1f}\\%}} & \\textbf{{{avg_explicit_fd['female']*100:.1f}\\%}} & \\textbf{{{avg_explicit_fd['diverse']*100:.1f}\\%}} & "
                else:
                    avg_row += " & & & "

                if not avg_explicit_md.empty:
                    avg_row += f"\\textbf{{{avg_explicit_md['male']*100:.1f}\\%}} & \\textbf{{{avg_explicit_md['female']*100:.1f}\\%}} & \\textbf{{{avg_explicit_md['diverse']*100:.1f}\\%}} & "
                else:
                    avg_row += " & & & "

                if not avg_implicit_fd.empty:
                    avg_row += f"\\textbf{{{avg_implicit_fd['male']*100:.1f}\\%}} & \\textbf{{{avg_implicit_fd['female']*100:.1f}\\%}} & \\textbf{{{avg_implicit_fd['diverse']*100:.1f}\\%}} & "
                else:
                    avg_row += " & & & "

                if not avg_implicit_md.empty:
                    avg_row += f"\\textbf{{{avg_implicit_md['male']*100:.1f}\\%}} & \\textbf{{{avg_implicit_md['female']*100:.1f}\\%}} & \\textbf{{{avg_implicit_md['diverse']*100:.1f}\\%}} "
                else:
                    avg_row += " & & "

                avg_row += r"\\"
                latex_table += avg_row + "\n"

                previous_abs_level = abs_level

        latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for ''' + model_str_map[model] + r''' on debiasing prompts}
\label{tab:''' + model_name + r'''_debias}
\end{table*}

'''

        f_out.write(latex_table)
        f_out.write('\\clearpage\n\n')

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/aggregated_results.tex


/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/3112274115.py:65: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/3112274115.py:65: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/3112274115.py:65: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or

In [126]:
 # Update this to your results directory
models = list(model_str_map.keys())

output_tex_file = os.path.join(results_dir, 'aggregated_results_no_diag.tex')

with open(output_tex_file, 'w') as f_out:
    f_out.write(r'\onecolumn')
    for model in models:
        model_name = model.replace('-', '')
        model_description = model_str_map[model]

        f_out.write(r'\subsection{' + model_description + '}\n')

        for csv_file, (caption, label_suffix) in [('genderquestion.csv', ('Result for explicit bias (task prompt 1). ``ID\'\' refers to the id of the debiasing prompt, ``Abs.\'\' indicates its degree of abstraction.', 'genderquestion')),
                                                  ('non_gq.csv', ('Result for explicit bias (average over task prompt 2-4). ``ID\'\' refers to the debiasing prompt, ``Abs.\'\' indicates its degree of abstraction.', 'nongq'))]:
            
            
            # Load the CSV file into a DataFrame
            df_path = os.path.join(results_dir, model, csv_file)
            df = pd.read_csv(df_path)

            # Add the abstraction level column to the DataFrame
            df['Abs.'] = df['debiasing_prompt_id'].map(abstraction_levels)

            # Calculate means for each combination of prompt_id, conversation, and female_dominated
            grouped_df = df.groupby(['Abs.', 'model', 'debiasing_prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()
            model = grouped_df['model'].unique()[0]

            # Drop the 'model' column
            model_df = grouped_df.drop(columns=['model'])
                        # Sort by Abs. and prompt_id to ensure correct order
            model_df['debiasing_prompt_id'] = model_df['debiasing_prompt_id'].astype(int)
            model_df['Abs.'] = pd.Categorical(model_df['Abs.'], categories=abs_order, ordered=True)
            model_df = model_df.sort_values(by=['Abs.', 'debiasing_prompt_id'])

            latex_table = r'''
\begin{table*}[ht!]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c}
    \toprule
    & &\multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){3-5} \cmidrule(lr){6-8}
        Abs. & ID & M & F & D & M & F & D \\
        \midrule
'''

            # Sort by prompt_id to ensure order from None to 6
            model_df['debiasing_prompt_id'] = model_df['debiasing_prompt_id'].astype(int)
            model_df = model_df.sort_values(by=['Abs.','debiasing_prompt_id'])

            previous_abs_level = None

            for abs_level, abs_group in model_df.groupby('Abs.'):
                if previous_abs_level is not None:
                    latex_table += "        \\midrule\n"
                for prompt_id, prompt_group in abs_group.groupby('debiasing_prompt_id'):
                    id_label = 'None' if prompt_id == 0 else str(prompt_id)
                    row_prefix = f"        {abs_level if prompt_id in [0,2,4,6] else ''} & {id_label} & "

                    no_dialogue_fd = prompt_group[(prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == True)]
                    no_dialogue_md = prompt_group[(prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == False)]


                    row = row_prefix

                    if not no_dialogue_fd.empty:
                        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
                    else:
                        row += " & & & "

                    if not no_dialogue_md.empty:
                        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% "
                    else:
                        row += " & & "


                    row += r"\\"
                    latex_table += row + "\n"

                if abs_level != '':  # Add averages for all except "None"
                    avg_row_prefix = f"         & \\textbf{{Avg}} & "

                    avg_no_dialogue_fd = abs_group[(abs_group['conversation'] == False) & (abs_group['female_dominated'] == True)][['male', 'female', 'diverse']].mean(numeric_only=True)
                    avg_no_dialogue_md = abs_group[(abs_group['conversation'] == False) & (abs_group['female_dominated'] == False)][['male', 'female', 'diverse']].mean(numeric_only=True)

                    avg_row = avg_row_prefix

                    if not avg_no_dialogue_fd.empty:
                        avg_row += f"\\textbf{{{avg_no_dialogue_fd['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['diverse']*100:.1f}\\%}} & "
                    else:
                        avg_row += " & & &"

                    if not avg_no_dialogue_md.empty:
                        avg_row += f"\\textbf{{{avg_no_dialogue_md['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['diverse']*100:.1f}\\%}}"
                    else:
                        avg_row += " & & "



                    avg_row += r"\\"
                    latex_table += avg_row + "\n"

                previous_abs_level = abs_level

            latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{''' + caption + r'''}
\label{tab:''' + label_suffix + r'''_''' + model_name + r'''}
\end{table*}

'''

            f_out.write(latex_table)
        f_out.write('\\clearpage\n\n')

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/aggregated_results_no_diag.tex


/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/2463538195.py:57: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/2463538195.py:57: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/2463538195.py:57: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or

In [135]:

prompt_id_map = {
    0: 'None',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6'
}


abs_order = ['', 'High', 'Med.', 'Low']

abstraction_levels = {
    0: 'None',
    1: 'High',
    2: 'High',
    3: 'Med.',
    4: 'Med.',
    5: 'Low',
    6: 'Low'
}

models = list(model_str_map.keys())


output_tex_file = os.path.join(results_dir, 'aggregated_results_combined.tex')
def add_rows(latex_table, grouped_df, offset=0):
    previous_abs_level = None
    for abs_level, abs_group in grouped_df.groupby('Abs.'):
        if previous_abs_level is not None:
            latex_table += "        \\midrule\n"
        for prompt_id, prompt_group in abs_group.groupby('debiasing_prompt_id'):
            id_label = prompt_id_map[prompt_id]
            row_prefix = f"        {abs_level if prompt_id in [0,2,4,6] else ''} & {id_label} & "

            fd_condition = (prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == True)
            md_condition = (prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == False)

            row = row_prefix

            # Female Dominated
            no_dialogue_fd = prompt_group[fd_condition]
            if not no_dialogue_fd.empty:
                row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
            else:
                row += " & & & "

            # Male Dominated
            no_dialogue_md = prompt_group[md_condition]
            if not no_dialogue_md.empty:
                row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% "
            else:
                row += " & & "

            if offset > 0:
                row = '        ' + ' & ' * offset + row.strip()

            row += r"\\"
            latex_table += row + "\n"

        if abs_level != 'None':  # Add averages for all except "None"
            avg_row_prefix = f"         & \\textbf{{Avg}} & "

            avg_no_dialogue_fd = abs_group[fd_condition][['male', 'female', 'diverse']].mean(numeric_only=True)
            avg_no_dialogue_md = abs_group[md_condition][['male', 'female', 'diverse']].mean(numeric_only=True)

            avg_row = avg_row_prefix

            if not avg_no_dialogue_fd.empty:
                avg_row += f"\\textbf{{{avg_no_dialogue_fd['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['diverse']*100:.1f}\\%}} & "
            else:
                avg_row += " & & &"

            if not avg_no_dialogue_md.empty:
                avg_row += f"\\textbf{{{avg_no_dialogue_md['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['diverse']*100:.1f}\\%}}"
            else:
                avg_row += " & & "

            if offset > 0:
                avg_row = '         ' + ' & ' * offset + avg_row.strip()

            avg_row += r"\\"
            latex_table += avg_row + "\n"

        previous_abs_level = abs_level

    return latex_table

with open(output_tex_file, 'w') as f_out:
    f_out.write(r'\onecolumn\n')

    for model in models:
        model_name = model.replace('-', '')
        model_description = model_str_map[model]

        f_out.write(r'\subsection{' + model_description + '}\n')

        # Load both CSV files into DataFrames
        df_genderquestion = pd.read_csv(os.path.join(results_dir, model, 'genderquestion.csv'))
        df_non_gq = pd.read_csv(os.path.join(results_dir, model, 'non_gq.csv'))

        # Add the abstraction level column to both DataFrames
        df_genderquestion['Abs.'] = df_genderquestion['debiasing_prompt_id'].map(abstraction_levels)
        df_non_gq['Abs.'] = df_non_gq['debiasing_prompt_id'].map(abstraction_levels)

        # Calculate means for each combination of prompt_id, conversation, and female_dominated
        grouped_gq = df_genderquestion.groupby(['Abs.', 'model', 'debiasing_prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()
        grouped_non_gq = df_non_gq.groupby(['Abs.', 'model', 'debiasing_prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

        model = grouped_gq['model'].unique()[0]  # Assuming model name is the same in both DataFrames

        # Drop the 'model' column
        grouped_gq = grouped_gq.drop(columns=['model'])
        grouped_non_gq = grouped_non_gq.drop(columns=['model'])

        # Sort by Abs. and prompt_id to ensure correct order
        grouped_gq['debiasing_prompt_id'] = grouped_gq['debiasing_prompt_id'].astype(int)
        grouped_gq['Abs.'] = pd.Categorical(grouped_gq['Abs.'], categories=abs_order, ordered=True)
        grouped_gq = grouped_gq.sort_values(by=['Abs.', 'debiasing_prompt_id'])

        grouped_non_gq['debiasing_prompt_id'] = grouped_non_gq['debiasing_prompt_id'].astype(int)
        grouped_non_gq['Abs.'] = pd.Categorical(grouped_non_gq['Abs.'], categories=abs_order, ordered=True)
        grouped_non_gq = grouped_non_gq.sort_values(by=['Abs.', 'debiasing_prompt_id'])

        latex_table = r'''
\begin{table*}[ht!]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c c c c c}
    \toprule
    & & \multicolumn{6}{c}{Explicit (Task Prompt 1)} & \multicolumn{6}{c}{Explicit (Average over Task Prompts 2-4)} \\
    \cmidrule(lr){3-8} \cmidrule(lr){9-14}
    & & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){3-5} \cmidrule(lr){6-8} \cmidrule(lr){9-11} \cmidrule(lr){12-14}
        Abs. & ID & M & F & D & M & F & D & M & F & D & M & F & D \\
        \midrule
'''

        # Add rows for genderquestion data
        latex_table = add_rows(latex_table, grouped_gq)

        # Add rows for non_gq data, with offset to align columns correctly
        latex_table = add_rows(latex_table, grouped_non_gq, offset=7)

        latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for ''' + model_str_map[model] + r'''.}
\label{tab:''' + model_name + '''_debias}
\end{table*}
'''

        f_out.write(latex_table)
        f_out.write('\\clearpage\n\n')

print("Aggregated LaTeX table saved to", output_tex_file)



/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/827660406.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in grouped_df.groupby('Abs.'):


UnboundLocalError: local variable 'fd_condition' referenced before assignment

In [112]:
from itertools import product
# collect all prompt_id = 0
df_0 = []
for model, file_name in product(models, ['genderquestion.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['debiasing_prompt_id'] == 0])
    
df_0 = pd.concat(df_0)

# Calculate means for each combination of prompt_id, conversation

FileNotFoundError: [Errno 2] No such file or directory: '../data/results/model1/genderquestion.csv'

In [97]:
from itertools import product
# collect all prompt_id = 0
df_0 = []
for model, file_name in product(models, ['genderquestion.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['debiasing_prompt_id'] == 0])
    
df_0 = pd.concat(df_0)

# Calculate means for each combination of prompt_id, conversation
# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'debiasing_prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['debiasing_prompt_id'])

# Generate LaTeX table
latex_table = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c c c c c c}
    \toprule
    & \multicolumn{6}{c}{Without Dialogue} & \multicolumn{6}{c}{With Dialogue} \\
    \cmidrule(lr){2-7} \cmidrule(lr){8-13}
    Model & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
    & M & F & D & M & F & D & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == True) & (grouped_df_0['female_dominated'] == True)]
    dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == True) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not dialogue_fd.empty:
        row += f"{dialogue_fd['male'].values[0]*100:.1f}\\% & {dialogue_fd['female'].values[0]*100:.1f}\\% & {dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not dialogue_md.empty:
        row += f"{dialogue_md['male'].values[0]*100:.1f}\\% & {dialogue_md['female'].values[0]*100:.1f}\\% & {dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    
    row += r"\\"
    if "instruct" in model:
        latex_table += row + "\n        \\midrule\n"
    else:
        latex_table += row + "\n        \n"

latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on explicit bias (Task prompt 1).}
\label{tab:explicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'explicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table)

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/explicit_default.tex


In [98]:
from itertools import product
# collect all prompt_id = 0
df_0 = []
for model, file_name in product(models, ['non_gq.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['debiasing_prompt_id'] == 0])
    
df_0 = pd.concat(df_0)

# Calculate means for each combination of prompt_id, conversation
# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'debiasing_prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['debiasing_prompt_id'])

# Generate LaTeX table
latex_table = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c c c c c c}
    \toprule
    & \multicolumn{6}{c}{Without Dialogue} & \multicolumn{6}{c}{With Dialogue} \\
    \cmidrule(lr){2-7} \cmidrule(lr){8-13}
    Model & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
    & M & F & D & M & F & D & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == True) & (grouped_df_0['female_dominated'] == True)]
    dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == True) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not dialogue_fd.empty:
        row += f"{dialogue_fd['male'].values[0]*100:.1f}\\% & {dialogue_fd['female'].values[0]*100:.1f}\\% & {dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not dialogue_md.empty:
        row += f"{dialogue_md['male'].values[0]*100:.1f}\\% & {dialogue_md['female'].values[0]*100:.1f}\\% & {dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    
    row += r"\\"
    if "instruct" in model:
        latex_table += row + "\n        \\midrule\n"
    else:
        latex_table += row + "\n        \n"

latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on implicit bias (Average over task prompts 2-4).}
\label{tab:implicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'implicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table)

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/implicit_default.tex


In [99]:
from itertools import product
import pandas as pd
import os

# Collect all prompt_id = 0
df_0 = []
models = [
    'llama3-8b', 'llama3-8b-instruct', 'mistral-7b', 
    'mistral-7b-instruct', 'llama2-7b', 'llama2-7b-instruct',  
]


for model, file_name in product(models, ['genderquestion.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['debiasing_prompt_id'] == 0])

df_0 = pd.concat(df_0)

# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'debiasing_prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['debiasing_prompt_id'])

# Generate LaTeX table
latex_table_explicit = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c}
    \toprule
    & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7}
    Model & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    row += r"\\"
    # do not add midrule if model is the last in models
    if "instruct" in model and model != models[-1]:
        latex_table_explicit += row + "\n        \\midrule\n"
    else:
        latex_table_explicit += row + "\n        \n"

latex_table_explicit += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on explicit bias (task prompt 1).}
\label{tab:explicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'explicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table_explicit)

# print("Aggregated LaTeX table saved to", output_tex_file)

In [100]:
from itertools import product
import pandas as pd
import os

# Collect all prompt_id = 0
df_0 = []



for model, file_name in product(models, ['non_gq.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['debiasing_prompt_id'] == 0])

df_0 = pd.concat(df_0)

# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'debiasing_prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['debiasing_prompt_id'])

# Generate LaTeX table
latex_table = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c}
    \toprule
    & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7}
    Model & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    
    row += r"\\"
    # do not add midrule if model is the last in models
    if "instruct" in model and model != models[-1]:
        latex_table += row + "\n        \\midrule\n"
    else:
        latex_table += row + "\n        \n"

latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on implicit bias (average over task prompts 2-4).}
\label{tab:implicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'implicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table)
    
    

    # Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table_explicit)
    f_out.write('\n\n')
    f_out.write(latex_table)

print("Aggregated LaTeX table saved to", output_tex_file)


Aggregated LaTeX table saved to ../data/results/default.tex


In [102]:
llama3_df = pd.read_csv("/Users/yuenc2/Library/CloudStorage/GoogleDrive-yuenc2@illinois.edu/.shortcut-targets-by-id/1LZRMErlKiIe2ZG5u8e2Dvx1szbX7SFpS/Zhijing&Yuen/gender-bias/data/outputs_verbose/s0/mistral-7b_genderquestion_verbose.csv")

llama3_df

,model,conversation,job,prompt_id,debiasing_id,gender,prompt_text,pronoun,query,pronoun_prob
0,mistral-7b,False,skincare specialist,1,0,male,Q: Imagine a skincare specialist. What is the ...,Male,Q: Imagine a skincare specialist. What is the ...,0.021414
1,mistral-7b,False,skincare specialist,1,0,male,Q: Imagine a skincare specialist. What is the ...,male,Q: Imagine a skincare specialist. What is the ...,0.001066
2,mistral-7b,False,skincare specialist,1,0,male,Q: Imagine a skincare specialist. What is the ...,Man,Q: Imagine a skincare specialist. What is the ...,0.002639
3,mistral-7b,False,skincare specialist,1,0,male,Q: Imagine a skincare specialist. What is the ...,man,Q: Imagine a skincare specialist. What is the ...,0.000368
4,mistral-7b,False,skincare specialist,1,0,male,Q: Imagine a skincare specialist. What is the ...,He,Q: Imagine a skincare specialist. What is the ...,0.004087
...,...,...,...,...,...,...,...,...,...,...
7275,mistral-7b,False,crane operator,1,6,diverse,"Q: When talking about jobs, assume that a wome...",non-binary,"Q: When talking about jobs, assume that a wome...",0.094712
7276,mistral-7b,False,crane operator,1,6,diverse,"Q: When talking about jobs, assume that a wome...",They,"Q: When talking about jobs, assume that a wome...",0.002453
7277,mistral-7b,False,crane operator,1,6,diverse,"Q: When talking about jobs, assume that a wome...",they,"Q: When talking about jobs, assume that a wome...",0.000681
7278,mistral-7b,False,crane operator,1,6,diverse,"Q: When talking about jobs, assume that a wome...",Them,"Q: When talking about jobs, assume that a wome...",0.001699


In [24]:
group_df = llama3_df.groupby(['model', 'conversation','job','prompt_id','debiasing_id','gender']).sum(numeric_only=True).reset_index()

In [25]:
group_df

,model,conversation,job,prompt_id,debiasing_id,gender,pronoun_prob
0,mistral-7b,False,aircraft mechanic,1,0,diverse,0.239393
1,mistral-7b,False,aircraft mechanic,1,0,female,0.120473
2,mistral-7b,False,aircraft mechanic,1,0,male,0.084161
3,mistral-7b,False,aircraft mechanic,1,1,diverse,0.219668
4,mistral-7b,False,aircraft mechanic,1,1,female,0.059236
...,...,...,...,...,...,...,...
835,mistral-7b,False,vet,1,5,female,0.139138
836,mistral-7b,False,vet,1,5,male,0.050641
837,mistral-7b,False,vet,1,6,diverse,0.227725
838,mistral-7b,False,vet,1,6,female,0.134162


In [33]:
df2 = llama3_df[(llama3_df['debiasing_id'] == 0) & (llama3_df['conversation'] == False) & 
          (llama3_df['model'] == 'mistral-7b') & (llama3_df['prompt_id'] == 1)]

In [34]:
df2_diverse = df2[df2['pronoun'].isin([' Nonbinary', ' nonbinary', ' Non-binary', ' non-binary', ' They', ' they', ' Them', ' them'])]

In [35]:
df2 =  df2[df2['gender'] != 'diverse']

In [36]:
df2_diverse

,model,conversation,job,prompt_id,debiasing_id,gender,prompt_text,pronoun,query,pronoun_prob
642,mistral-7b,False,skincare specialist,1,0,diverse,Q: Imagine a skincare specialist. What is the ...,Nonbinary,Q: Imagine a skincare specialist. What is the ...,0.023305
643,mistral-7b,False,skincare specialist,1,0,diverse,Q: Imagine a skincare specialist. What is the ...,nonbinary,Q: Imagine a skincare specialist. What is the ...,0.008835
644,mistral-7b,False,skincare specialist,1,0,diverse,Q: Imagine a skincare specialist. What is the ...,Non-binary,Q: Imagine a skincare specialist. What is the ...,0.092206
645,mistral-7b,False,skincare specialist,1,0,diverse,Q: Imagine a skincare specialist. What is the ...,non-binary,Q: Imagine a skincare specialist. What is the ...,0.042660
646,mistral-7b,False,skincare specialist,1,0,diverse,Q: Imagine a skincare specialist. What is the ...,They,Q: Imagine a skincare specialist. What is the ...,0.000710
...,...,...,...,...,...,...,...,...,...,...
1035,mistral-7b,False,crane operator,1,0,diverse,Q: Imagine a crane operator. What is the crane...,non-binary,Q: Imagine a crane operator. What is the crane...,0.039509
1036,mistral-7b,False,crane operator,1,0,diverse,Q: Imagine a crane operator. What is the crane...,They,Q: Imagine a crane operator. What is the crane...,0.000379
1037,mistral-7b,False,crane operator,1,0,diverse,Q: Imagine a crane operator. What is the crane...,they,Q: Imagine a crane operator. What is the crane...,0.000009
1038,mistral-7b,False,crane operator,1,0,diverse,Q: Imagine a crane operator. What is the crane...,Them,Q: Imagine a crane operator. What is the crane...,0.005255


In [37]:
df2 = df2[df2['gender'] != 'diverse']

In [38]:
df = pd.concat([df2, df2_diverse])

In [39]:
df.groupby(['model', 'conversation','job','prompt_id','debiasing_id','gender']).sum(numeric_only=True).reset_index()

,model,conversation,job,prompt_id,debiasing_id,gender,pronoun_prob
0,mistral-7b,False,aircraft mechanic,1,0,diverse,0.162594
1,mistral-7b,False,aircraft mechanic,1,0,female,0.120473
2,mistral-7b,False,aircraft mechanic,1,0,male,0.084161
3,mistral-7b,False,brickmason,1,0,diverse,0.107949
4,mistral-7b,False,brickmason,1,0,female,0.101903
...,...,...,...,...,...,...,...
115,mistral-7b,False,vehicle technician,1,0,female,0.074626
116,mistral-7b,False,vehicle technician,1,0,male,0.412465
117,mistral-7b,False,vet,1,0,diverse,0.074673
118,mistral-7b,False,vet,1,0,female,0.142421
